# explore how to recreate a coadd

trying to help with https://community.lsst.org/t/coadding-dp0-calexps-from-different-detectors-for-the-same-deepcoadd/6436

## bunch of imports

In [1]:
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import time
import os

import pandas
pandas.set_option('display.max_rows', 1000)

import lsst.daf.butler

import lsst.geom
# import lsst.afw.display as afwDisplay
# afwDisplay.setDefaultBackend('matplotlib')

# from astropy.wcs import WCS
# from astropy import units as u
# from astropy.coordinates import SkyCoord

## butler set up

In [2]:
repo = 's3://butler-us-central1-dp01'  
collection='2.2i/runs/DP0.1'
butler = lsst.daf.butler.Butler(repo, collections=collection)

## figure out _what_ you want to coadd

just use a few visits at a certain sky location

In [3]:
ra, dec = 55.064, -29.783
print(ra, dec)

55.064 -29.783


In [4]:
radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
print(radec)

(55.0640000000, -29.7830000000)


In [5]:
skymap = butler.get('skyMap')

In [6]:
tractInfo = skymap.findTract(radec)
tract = tractInfo.getId()
patchInfo = tractInfo.findPatch(radec)
patch = tractInfo.getSequentialPatchIndex(patchInfo)
print(tract,patch)

4638 43


In [7]:
dataId = {'band': 'i', 'tract': tract, 'patch': patch}
deepCoadd_calexp = butler.get('deepCoadd_calexp', dataId=dataId)

In [8]:
coadd_inputs = butler.get("deepCoadd_calexp.coaddInputs", dataId)
coadd_inputs_ccds_table = coadd_inputs.ccds.asAstropy()

use_detector = coadd_inputs_ccds_table['ccd'][0]
use_visit = coadd_inputs_ccds_table['visit'][0]

# calexp = butler.get('calexp', detector=use_detector, visit=use_visit)

In [9]:
all_visits = np.asarray(coadd_inputs_ccds_table['visit'], dtype='int')
unique_visits = np.unique( np.sort( all_visits ) )

indices = [0,1,2,3,4]
print('The first five unique visits: ', unique_visits[indices])

The first five unique visits:  [174549 174602 204556 204559 204560]


## Try using command line tasks

Get help from
 * https://pipelines.lsst.io/getting-started/coaddition.html
 * https://hackmd.io/@lsk/merian#A-Useful-Commands
 * https://pipelines.lsst.io/middleware/faq.html

In [11]:
os.system('setup lsst_distrib')

0

In [12]:
# os.system('ls ${PIPE_TASKS_DIR}')

In [13]:
# ! ls ${PIPE_TASKS_DIR}

In [14]:
os.system('mkdir mywarps')

0

In [15]:
### arguments that failed

# -d "tract = 4638 AND skymap = 'DC2' AND patch = 43 AND visit in (174549, 174602, 204556)" \
# -d "instrument='LSSTCam-imSim' AND band='i' AND tract = 4638 AND skymap = 'DC2' AND patch = 43" \
# -d "instrument='LSSTCam-imSim' AND skymap = 'DC2' AND visit in (174549, 174602, 204556)" \


### arguments I changed how passed, seemed to have no effect
# --instrument lsst.obs.lsst.LsstCamImSim  \


In [16]:
! pipetask run -b "s3://butler-us-central1-dp01/butler.yaml" \
               -i "2.2i/runs/DP0.1" \
               -d "instrument='LSSTCam-imSim' AND skymap = 'DC2' AND tract = 4638 AND patch = 43" \
               -p ${PIPE_TASKS_DIR}/pipelines/DRP.yaml#makeWarp \
               -o "mywarps/" \
               --register-dataset-types

lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/runs/DP0.1/coadd/wfd/dr6/v1'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/runs/DP0.1/coadd/wfd/dr6/v1/u'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/runs/DP0.1/coadd/wfd/dr6/v1/grizy'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/runs/DP0.1/calexp/v1'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/raw/DP0'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/calib/PREOPS-301'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/calib/gen2'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection '2.2i/calib/PREOPS-301/unbounded'.
lsst.pipe.base.graphBuilder WARNING: No datasets of type visitSummary in collection 'ref

In [ ]:
# ! pipetask qgraph -h

In [ ]:
# ! pipetask run -h

## Try importing the packages and avoid using command-line tasks

https://pipelines.lsst.io/modules/lsst.pipe.tasks/index.html#lsst-pipe-tasks-command-line-tasks

Jeff managed to do this for image differencing in https://github.com/rubin-dp0/delegate-contributions-dp01/blob/u/jcarlin/diff_im/diff_im/diff_im_test.ipynb

Yusra shows use of tasks and configs in https://github.com/LSSTScienceCollaborations/StackClubCourse/blob/master/Session02/IntroToDataProductsAndTasks.ipynb

In [10]:
from lsst.pipe.tasks.warpAndPsfMatch import WarpAndPsfMatchTask
from lsst.pipe.tasks.assembleCoadd import AssembleCoaddTask

In [17]:
config = WarpAndPsfMatchTask.ConfigClass